# Industrial Water Demand

This notebook removes enterprises from the data which doesn't use water as an input into production. we use information from Block H to furtehr subset information in Block J


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
dir_path = Path.cwd().parents[1]

data_folder = dir_path.joinpath("data/asi/raw")

nic_path = dir_path.joinpath("data/asi/external/nic.parquet")

npcms_path = dir_path.joinpath("data/asi/external/npcms_2011_revised.xls")

valid_output_marker_path = dir_path.joinpath("data/asi/external/multi_prods_valid_outputs.csv")

metric_converter_path = dir_path.joinpath("data/asi/external/asi_output_units.xlsx")


def raw_file_path_make(block_name: str):
    return data_folder.joinpath(f"blk{block_name}201920.sav")

## Block Level Data Cleaning

### Block J - Production and By products

Isolating quantity manufactured from the frames


In [4]:
blockJ_path = raw_file_path_make("J")

blkJ = pd.read_spss(blockJ_path)
blkJ

,yr,blk,AJ01,J11,J13,J14,J15,J16,J17,J18,J19,J110,J111,J112,J113
0,20,J,100002,1.0,0325003,27.0,245.89,243.3,41772980.0,0.0,0.0,0.0,0.0,171693.30,42217666.0
1,20,J,100002,11.0,9921100,0.0,0.00,0.0,1885510.0,0.0,0.0,0.0,0.0,0.00,1885510.0
2,20,J,100002,12.0,9995000,0.0,0.00,0.0,43658490.0,0.0,0.0,0.0,0.0,0.00,44103176.0
3,20,J,100003,1.0,0325003,27.0,366.96,48.7,10538000.0,0.0,0.0,355669.0,0.0,209082.77,76725014.0
4,20,J,100003,12.0,9995000,0.0,0.00,0.0,10538000.0,0.0,0.0,355669.0,0.0,0.00,76725014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133874,20,J,223175,12.0,9995000,0.0,0.00,0.0,3183639.0,0.0,0.0,486860.0,0.0,0.00,4854926.0
133875,20,J,223176,1.0,4482300,15.0,3971.00,3971.0,32683409.0,1662058.0,0.0,320192.0,0.0,7731.34,30701159.0
133876,20,J,223176,12.0,9995000,0.0,0.00,0.0,32683409.0,1662058.0,0.0,320192.0,0.0,0.00,30701159.0
133877,20,J,223204,1.0,2227001,9.0,326862.00,326862.0,858420.0,0.0,0.0,0.0,0.0,2.63,858420.0


In [5]:
blkJ.drop(["yr", "blk"], axis=1, inplace=True)
blkJ

,AJ01,J11,J13,J14,J15,J16,J17,J18,J19,J110,J111,J112,J113
0,100002,1.0,0325003,27.0,245.89,243.3,41772980.0,0.0,0.0,0.0,0.0,171693.30,42217666.0
1,100002,11.0,9921100,0.0,0.00,0.0,1885510.0,0.0,0.0,0.0,0.0,0.00,1885510.0
2,100002,12.0,9995000,0.0,0.00,0.0,43658490.0,0.0,0.0,0.0,0.0,0.00,44103176.0
3,100003,1.0,0325003,27.0,366.96,48.7,10538000.0,0.0,0.0,355669.0,0.0,209082.77,76725014.0
4,100003,12.0,9995000,0.0,0.00,0.0,10538000.0,0.0,0.0,355669.0,0.0,0.00,76725014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133874,223175,12.0,9995000,0.0,0.00,0.0,3183639.0,0.0,0.0,486860.0,0.0,0.00,4854926.0
133875,223176,1.0,4482300,15.0,3971.00,3971.0,32683409.0,1662058.0,0.0,320192.0,0.0,7731.34,30701159.0
133876,223176,12.0,9995000,0.0,0.00,0.0,32683409.0,1662058.0,0.0,320192.0,0.0,0.00,30701159.0
133877,223204,1.0,2227001,9.0,326862.00,326862.0,858420.0,0.0,0.0,0.0,0.0,2.63,858420.0


In [6]:
blkJ.rename(
    columns={
        "AJ01": "dsl",
        "J11": "output_sl_no",
        "J13": "output_item_code",
        "J14": "output_unit_qty_code",
        "J15": "output_qty_manuf",
        "J16": "output_qty_sold",
        "J17": "gross_sales_value",
        "J18": "gst",
        "J19": "other_tax",
        "J110": "other_distributive_exp",
        "J111": "subsidy",
        "J112": "per_unit_net_sales_value",
        "J113": "ex_factory_value",
    },
    inplace=True,
)
blkJ

,dsl,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value
0,100002,1.0,0325003,27.0,245.89,243.3,41772980.0,0.0,0.0,0.0,0.0,171693.30,42217666.0
1,100002,11.0,9921100,0.0,0.00,0.0,1885510.0,0.0,0.0,0.0,0.0,0.00,1885510.0
2,100002,12.0,9995000,0.0,0.00,0.0,43658490.0,0.0,0.0,0.0,0.0,0.00,44103176.0
3,100003,1.0,0325003,27.0,366.96,48.7,10538000.0,0.0,0.0,355669.0,0.0,209082.77,76725014.0
4,100003,12.0,9995000,0.0,0.00,0.0,10538000.0,0.0,0.0,355669.0,0.0,0.00,76725014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133874,223175,12.0,9995000,0.0,0.00,0.0,3183639.0,0.0,0.0,486860.0,0.0,0.00,4854926.0
133875,223176,1.0,4482300,15.0,3971.00,3971.0,32683409.0,1662058.0,0.0,320192.0,0.0,7731.34,30701159.0
133876,223176,12.0,9995000,0.0,0.00,0.0,32683409.0,1662058.0,0.0,320192.0,0.0,0.00,30701159.0
133877,223204,1.0,2227001,9.0,326862.00,326862.0,858420.0,0.0,0.0,0.0,0.0,2.63,858420.0


Adding unit qty codes

In [7]:
unit_dict = {
    1: "Bags",
    3: "Cubic Meter",
    4: "Carat",
    6: "Gramme",
    7: "K.Litres & Th. Litres",
    8: "Km",
    9: "Kg",
    12: "Litres",
    13: "Megawatt",
    14: "Metres",
    15: "Nos.",
    16: "Pair",
    17: "Ream",
    18: "Roll",
    20: "Sq. Metre",
    22: "Th. Nos",
    23: "Th. Cubic Metre",
    24: "Th. K. Litre",
    25: "Th. Pair",
    27: "Tonne",
    28: "K. Watt",
    31: "Feet",
    38: "Cubic Cm"
}


In [8]:
blkJ['output_unit_qty_code'] = blkJ['output_unit_qty_code'].map(unit_dict)

In [9]:
blkJ['output_unit_qty_code'].value_counts(dropna=False)

output_unit_qty_code
NaN                      55137
Tonne                    26913
Nos.                     20279
Kg                       16208
Th. Nos                   6018
Metres                    2267
Litres                    2010
Sq. Metre                 1183
Cubic Meter               1159
K.Litres & Th. Litres      797
Pair                       685
Gramme                     339
K. Watt                    273
Megawatt                   188
Feet                       170
Carat                       90
Th. Cubic Metre             54
Roll                        51
Km                          21
Th. K. Litre                21
Bags                        10
Th. Pair                     4
Ream                         2
Name: count, dtype: int64

In block A, there is information about 66853 units. However, Block J has only 45708 units

In [10]:
blkJ['dsl'].nunique()

45708

Removing by products and total production from the dataframe

In [11]:
blkJ = blkJ[~blkJ['output_item_code'].isin(['9921100', '9995000'])]
blkJ

,dsl,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value
0,100002,1.0,0325003,Tonne,245.89,243.30,4.177298e+07,0.0,0.0,0.0,0.0,171693.30,4.221767e+07
3,100003,1.0,0325003,Tonne,366.96,48.70,1.053800e+07,0.0,0.0,355669.0,0.0,209082.77,7.672501e+07
5,100005,1.0,0325003,Tonne,83.00,70.00,8.161923e+06,0.0,0.0,0.0,0.0,116598.90,9.677709e+06
7,100006,1.0,1710003,Megawatt,1091170.64,1091170.64,3.769026e+09,0.0,0.0,4749436.0,0.0,3449.76,3.764277e+09
9,100007,1.0,1710003,Megawatt,1975547.90,1975547.90,6.815166e+09,0.0,0.0,0.0,0.0,3449.76,6.815166e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133869,223173,2.0,3823009,Carat,114976.35,114976.35,7.439861e+07,0.0,0.0,0.0,0.0,647.08,7.439861e+07
133871,223174,1.0,3824002,Gramme,500119.00,500119.00,2.090873e+09,154879453.0,0.0,1447993.0,0.0,3868.17,1.934545e+09
133873,223175,1.0,4391401,Nos.,4736.79,2631.16,3.183639e+06,0.0,0.0,486860.0,0.0,1024.94,4.854926e+06
133875,223176,1.0,4482300,Nos.,3971.00,3971.00,3.268341e+07,1662058.0,0.0,320192.0,0.0,7731.34,3.070116e+07


### Block H - Water Inputs and Usage

Using the codes obtained from NPCMS 2011 (revised) we have isolated codes for water input. Value and quantity of water input per unti of production will be calculated usin information Block H


In [12]:
blockH_path = raw_file_path_make("H")

blkH = pd.read_spss(blockH_path).drop(['yr', 'blk'], axis=1)
blkH.rename(columns={
     'AH01':"dsl", 'H11':"input_sl_no", 'H13':"input_npcms_item_code", 'H14':"input_unit_qty_code", 'H15':"input_qty_consumed", 'H16':"input_purchase_value", 'H17':"input_rate_per_unit"
}, inplace=True)

blkH

,dsl,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit
0,100002,20.0,9920400,0.0,0.00,844280.0,0.00
1,100002,21.0,9922000,0.0,0.00,937875.0,0.00
2,100002,14.0,9990800,0.0,0.00,20272.0,0.00
3,100002,16.0,9990500,28.0,479084.68,1844476.0,3.85
4,100002,17.0,9990600,0.0,0.00,90250.0,0.00
...,...,...,...,...,...,...,...
555091,223219,23.0,9993000,0.0,0.00,2274610.0,0.00
555092,223220,16.0,9990500,28.0,1523913.00,12191304.0,8.00
555093,223220,17.0,9990600,0.0,0.00,3000000.0,0.00
555094,223220,22.0,9992000,0.0,0.00,15191304.0,0.00


In [13]:
blkH['input_unit_qty_code'] = blkH['input_unit_qty_code'].map(unit_dict)
blkH

,dsl,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit
0,100002,20.0,9920400,NaN,0.00,844280.0,0.00
1,100002,21.0,9922000,NaN,0.00,937875.0,0.00
2,100002,14.0,9990800,NaN,0.00,20272.0,0.00
3,100002,16.0,9990500,K. Watt,479084.68,1844476.0,3.85
4,100002,17.0,9990600,NaN,0.00,90250.0,0.00
...,...,...,...,...,...,...,...
555091,223219,23.0,9993000,NaN,0.00,2274610.0,0.00
555092,223220,16.0,9990500,K. Watt,1523913.00,12191304.0,8.00
555093,223220,17.0,9990600,NaN,0.00,3000000.0,0.00
555094,223220,22.0,9992000,NaN,0.00,15191304.0,0.00


In [14]:
water_codes = [
    "1730001",
    "1730002",
    "1730099",
    "1740000",
    "1800001",
    "1800002",
    "1800099",
]

water_codes_dict ={
    "1730001":"Hot water",
    "1730002":"Steam",
    "1730099":"Steam and hot water nec",
    "1740000":"Ice and snow",
    "1800001":"Water for industrial use",
    "1800002":"Water for domestic use",
    "1800099":"Natural water nec"}

Please note:

- All inputs except 1740000 is measured in Th. K. Liters. This has been converted to cubic meters by multiplying it with 1000.

- NPCMS code 1740000 stands for Ice and Snow which is measured in tonnes. So we convert it to cubic meters by multiplying the quantity consumed by 2.831685

In [15]:
blkH_water = blkH[blkH['input_npcms_item_code'].isin(water_codes)]

# making all water quantities in cubic meters
blkH_water['input_qty_consumed'] = np.where( blkH_water['input_npcms_item_code'] != '1740000', blkH_water['input_qty_consumed']* 1000, blkH_water['input_qty_consumed'])

#converting ice measured in toinnes to cubic meters
blkH_water['input_qty_consumed'] = np.where( blkH_water['input_npcms_item_code'] == '1740000', blkH_water['input_qty_consumed']* 2.831685, blkH_water['input_qty_consumed'])

#adding string values to input uniut qty code

blkH_water['input_npcms_item_code'] = blkH_water['input_npcms_item_code'].map(water_codes_dict)

blkH_water



/tmp/ipykernel_97178/934858272.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blkH_water['input_qty_consumed'] = np.where( blkH_water['input_npcms_item_code'] != '1740000', blkH_water['input_qty_consumed']* 1000, blkH_water['input_qty_consumed'])
/tmp/ipykernel_97178/934858272.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blkH_water['input_qty_consumed'] = np.where( blkH_water['input_npcms_item_code'] == '1740000', blkH_water['input_qty_consumed']* 2.831685, blkH_water['input_qty_consumed'])
/tmp

,dsl,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit
27,100006,1.0,Natural water nec,Th. K. Litre,3.974902e+09,118094328.0,29.71
131,100016,4.0,Water for industrial use,Th. K. Litre,3.282010e+06,2545623.0,775.63
675,100078,4.0,Natural water nec,Th. K. Litre,1.163322e+07,345623.0,29.71
744,100085,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71
766,100087,5.0,Water for domestic use,Th. K. Litre,8.093000e+04,50000.0,617.84
...,...,...,...,...,...,...,...
549383,221805,5.0,Water for industrial use,Th. K. Litre,4.268996e+08,100680000.0,235.84
550834,222173,1.0,Natural water nec,Th. K. Litre,8.246380e+06,245000.0,29.71
550844,222175,1.0,Natural water nec,Th. K. Litre,4.469379e+08,13278524.0,29.71
550855,222176,1.0,Natural water nec,Th. K. Litre,8.799700e+05,100000.0,113.64


There are 4 cases where the water based input is zero. 

In [16]:
blkH_water[blkH_water['input_qty_consumed']==0]

,dsl,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit
96724,110687,2.0,Water for industrial use,Th. K. Litre,0.0,1.0,235.84
96743,110689,2.0,Water for industrial use,Th. K. Litre,0.0,1.0,235.84
151861,117314,3.0,Water for industrial use,Th. K. Litre,0.0,1.0,235.84
451160,205714,2.0,Water for industrial use,Th. K. Litre,0.0,1.0,235.84


In [17]:
# blkH_water['input_qty_consumed'] = np.where(
#     blkH_water['input_qty_consumed'] == 0, 
#     np.nan, 
#     blkH_water['input_qty_consumed']
# )

# blkH_water[blkH_water['input_qty_consumed']==0]

In [18]:
blkH_water

,dsl,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit
27,100006,1.0,Natural water nec,Th. K. Litre,3.974902e+09,118094328.0,29.71
131,100016,4.0,Water for industrial use,Th. K. Litre,3.282010e+06,2545623.0,775.63
675,100078,4.0,Natural water nec,Th. K. Litre,1.163322e+07,345623.0,29.71
744,100085,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71
766,100087,5.0,Water for domestic use,Th. K. Litre,8.093000e+04,50000.0,617.84
...,...,...,...,...,...,...,...
549383,221805,5.0,Water for industrial use,Th. K. Litre,4.268996e+08,100680000.0,235.84
550834,222173,1.0,Natural water nec,Th. K. Litre,8.246380e+06,245000.0,29.71
550844,222175,1.0,Natural water nec,Th. K. Litre,4.469379e+08,13278524.0,29.71
550855,222176,1.0,Natural water nec,Th. K. Litre,8.799700e+05,100000.0,113.64


Also, thousand liters is 1 cubic meter.

We have 493 units using water inputs

In [19]:
blkH_water['dsl'].nunique()

493

Out of the above 493 units, no firm uses more than 1 water based input for production. So each observation is a unique entry in Block H 

In [20]:
blkH_water.groupby('dsl')['input_npcms_item_code'].count().reset_index()['input_npcms_item_code'].value_counts()

input_npcms_item_code
1    493
Name: count, dtype: int64

Checking for natural water use which doesn't have an economic cost

In [21]:
blkH_water[blkH_water['input_npcms_item_code']=="1800099"]

,dsl,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit


### Merging J and H

In [22]:
waterJH = pd.merge(blkJ, blkH_water, on ="dsl", validate='m:1', how='right')
waterJH

,dsl,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit
0,100006,1.0,1710003,Megawatt,1091170.64,1091170.64,3.769026e+09,0.0,0.0,4749436.0,0.0,3449.76,3.764277e+09,1.0,Natural water nec,Th. K. Litre,3.974902e+09,118094328.0,29.71
1,100016,1.0,2143999,Litres,3662469.99,3587556.00,1.173326e+08,0.0,0.0,6118329.0,0.0,31.00,1.135366e+08,4.0,Water for industrial use,Th. K. Litre,3.282010e+06,2545623.0,775.63
2,100078,1.0,2413101,Litres,592912.99,595394.00,3.215130e+07,0.0,0.0,3321700.0,0.0,48.42,2.870947e+07,4.0,Natural water nec,Th. K. Litre,1.163322e+07,345623.0,29.71
3,100085,4.0,2449002,Litres,5252280.00,5252280.00,1.050456e+08,0.0,569032.0,1638400.0,0.0,19.58,1.028382e+08,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71
4,100085,5.0,2441003,Litres,2458109.97,2350600.00,2.949493e+07,0.0,0.0,910983.0,0.0,12.16,2.989130e+07,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,221805,1.0,2227003,Kg,5426535.00,5330085.00,8.829286e+08,25716366.0,0.0,14449166.0,0.0,158.11,8.580132e+08,5.0,Water for industrial use,Th. K. Litre,4.268996e+08,100680000.0,235.84
696,222173,1.0,2441099,Litres,203059.81,203059.81,1.460000e+06,0.0,0.0,0.0,0.0,7.19,1.460000e+06,1.0,Natural water nec,Th. K. Litre,8.246380e+06,245000.0,29.71
697,222175,1.0,2441001,Litres,1864483.01,1678516.00,2.402531e+07,0.0,0.0,274298.0,0.0,14.15,2.638244e+07,1.0,Natural water nec,Th. K. Litre,4.469379e+08,13278524.0,29.71
698,222176,1.0,2441099,Litres,122461.75,122461.75,8.805000e+05,0.0,0.0,0.0,0.0,7.19,8.805000e+05,1.0,Natural water nec,Th. K. Litre,8.799700e+05,100000.0,113.64


Now within data frame we have 493 unique units which use water as an inout in its production process

In [23]:
waterJH['dsl'].nunique()

493

### Finding the main product produced
Each industrial unit must have a main product that its producing. However, ASI provides all products and by products produced by the unit in Block J and doesn't differentiate between them. So, we have to isolate all firms which prodcues more than 1 product. Then, from the remaining industrial units that produce multiple products, we must inspect and identify what type of industry it belongs what is expected to be it's main product.

Creating an indicator where more than 1 product is listed



In [24]:
waterJH.groupby('dsl')['output_unit_qty_code'].count().reset_index()['output_unit_qty_code'].value_counts(dropna=False)

output_unit_qty_code
1     378
2      44
3      22
0      21
4      13
5       8
6       2
7       2
10      2
9       1
Name: count, dtype: int64

According to the indicator, there are 378 firms which produces only one output and so on

In [25]:
prod_count = waterJH.groupby('dsl')['output_unit_qty_code'].count().reset_index().rename(columns={'output_unit_qty_code':'prod_count'})
prod_count

,dsl,prod_count
0,100006,1
1,100016,1
2,100078,1
3,100085,3
4,100087,2
...,...,...
488,221805,1
489,222173,1
490,222175,1
491,222176,1


In [26]:
waterJH = waterJH.merge(prod_count, on='dsl', how='left', validate='m:1')
waterJH['multi_prod_indic'] = np.where(waterJH['prod_count']==1, 0, 1)
waterJH

,dsl,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit,prod_count,multi_prod_indic
0,100006,1.0,1710003,Megawatt,1091170.64,1091170.64,3.769026e+09,0.0,0.0,4749436.0,0.0,3449.76,3.764277e+09,1.0,Natural water nec,Th. K. Litre,3.974902e+09,118094328.0,29.71,1,0
1,100016,1.0,2143999,Litres,3662469.99,3587556.00,1.173326e+08,0.0,0.0,6118329.0,0.0,31.00,1.135366e+08,4.0,Water for industrial use,Th. K. Litre,3.282010e+06,2545623.0,775.63,1,0
2,100078,1.0,2413101,Litres,592912.99,595394.00,3.215130e+07,0.0,0.0,3321700.0,0.0,48.42,2.870947e+07,4.0,Natural water nec,Th. K. Litre,1.163322e+07,345623.0,29.71,1,0
3,100085,4.0,2449002,Litres,5252280.00,5252280.00,1.050456e+08,0.0,569032.0,1638400.0,0.0,19.58,1.028382e+08,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,3,1
4,100085,5.0,2441003,Litres,2458109.97,2350600.00,2.949493e+07,0.0,0.0,910983.0,0.0,12.16,2.989130e+07,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,221805,1.0,2227003,Kg,5426535.00,5330085.00,8.829286e+08,25716366.0,0.0,14449166.0,0.0,158.11,8.580132e+08,5.0,Water for industrial use,Th. K. Litre,4.268996e+08,100680000.0,235.84,1,0
696,222173,1.0,2441099,Litres,203059.81,203059.81,1.460000e+06,0.0,0.0,0.0,0.0,7.19,1.460000e+06,1.0,Natural water nec,Th. K. Litre,8.246380e+06,245000.0,29.71,1,0
697,222175,1.0,2441001,Litres,1864483.01,1678516.00,2.402531e+07,0.0,0.0,274298.0,0.0,14.15,2.638244e+07,1.0,Natural water nec,Th. K. Litre,4.469379e+08,13278524.0,29.71,1,0
698,222176,1.0,2441099,Litres,122461.75,122461.75,8.805000e+05,0.0,0.0,0.0,0.0,7.19,8.805000e+05,1.0,Natural water nec,Th. K. Litre,8.799700e+05,100000.0,113.64,1,0


Since Block J only has subset of the actual 66K units, there are 21 units for which we dont have production information. We are removing these 21

In [27]:
waterJH[waterJH['prod_count']==0]

,dsl,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit,prod_count,multi_prod_indic
226,117817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,Ice and snow,Tonne,2.406932e+02,3.973200e+05,4674.35,0,1
234,119632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Water for industrial use,Th. K. Litre,1.391014e+07,3.280567e+06,235.84,0,1
314,124278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Water for domestic use,Th. K. Litre,1.965033e+09,1.214076e+09,617.84,0,1
409,132336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,Water for industrial use,Th. K. Litre,7.881600e+06,7.093440e+05,90.00,0,1
484,144006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,Water for industrial use,Th. K. Litre,6.473020e+06,6.227179e+06,962.02,0,1
499,144771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,Water for industrial use,Th. K. Litre,2.614420e+06,6.165850e+05,235.84,0,1
500,144802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Water for industrial use,Th. K. Litre,4.128370e+06,9.736340e+05,235.84,0,1
501,144805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,Water for industrial use,Th. K. Litre,3.443490e+06,8.121120e+05,235.84,0,1
502,144841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,Water for industrial use,Th. K. Litre,4.578350e+06,1.079758e+06,235.84,0,1
503,145538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,Water for industrial use,Th. K. Litre,4.407563e+07,1.039480e+07,235.84,0,1


In [28]:
waterJH = waterJH[waterJH['prod_count']!=0]
waterJH

,dsl,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit,prod_count,multi_prod_indic
0,100006,1.0,1710003,Megawatt,1091170.64,1091170.64,3.769026e+09,0.0,0.0,4749436.0,0.0,3449.76,3.764277e+09,1.0,Natural water nec,Th. K. Litre,3.974902e+09,118094328.0,29.71,1,0
1,100016,1.0,2143999,Litres,3662469.99,3587556.00,1.173326e+08,0.0,0.0,6118329.0,0.0,31.00,1.135366e+08,4.0,Water for industrial use,Th. K. Litre,3.282010e+06,2545623.0,775.63,1,0
2,100078,1.0,2413101,Litres,592912.99,595394.00,3.215130e+07,0.0,0.0,3321700.0,0.0,48.42,2.870947e+07,4.0,Natural water nec,Th. K. Litre,1.163322e+07,345623.0,29.71,1,0
3,100085,4.0,2449002,Litres,5252280.00,5252280.00,1.050456e+08,0.0,569032.0,1638400.0,0.0,19.58,1.028382e+08,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,3,1
4,100085,5.0,2441003,Litres,2458109.97,2350600.00,2.949493e+07,0.0,0.0,910983.0,0.0,12.16,2.989130e+07,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,221805,1.0,2227003,Kg,5426535.00,5330085.00,8.829286e+08,25716366.0,0.0,14449166.0,0.0,158.11,8.580132e+08,5.0,Water for industrial use,Th. K. Litre,4.268996e+08,100680000.0,235.84,1,0
696,222173,1.0,2441099,Litres,203059.81,203059.81,1.460000e+06,0.0,0.0,0.0,0.0,7.19,1.460000e+06,1.0,Natural water nec,Th. K. Litre,8.246380e+06,245000.0,29.71,1,0
697,222175,1.0,2441001,Litres,1864483.01,1678516.00,2.402531e+07,0.0,0.0,274298.0,0.0,14.15,2.638244e+07,1.0,Natural water nec,Th. K. Litre,4.469379e+08,13278524.0,29.71,1,0
698,222176,1.0,2441099,Litres,122461.75,122461.75,8.805000e+05,0.0,0.0,0.0,0.0,7.19,8.805000e+05,1.0,Natural water nec,Th. K. Litre,8.799700e+05,100000.0,113.64,1,0


Inorder to identify the main product we need two information, NIC division and NPCMS product name. 

NIC code can be brought in by first merging with Block A and NIC frame.

NPCMS can be brought in by merging with NPCMS data frame

### Block A

we isolate all necessary enterprise identifiers from the Block A file


In [29]:
blockA_path = raw_file_path_make("A")


blkA = pd.read_spss(blockA_path)
blkA

,yr,blk,a1,a2,a3,a4,a5,a7,a8,a9,a10,a11,a12,bonus,pf,welfare,mwdays,nwdays,wdays,costop,expshare,mult
0,20,A,100001,99999,1.0,9999,12007,01,99,2.0,9999,1.0,2.0,0.0,0.0,0.0,0.0,366.0,366.0,124152.0,0.0,1.00
1,20,A,100002,99999,1.0,9999,12007,01,99,2.0,9999,1.0,1.0,0.0,0.0,0.0,178.0,118.0,296.0,37870282.0,0.0,1.00
2,20,A,100003,99999,1.0,9999,12007,01,99,1.0,9999,1.0,1.0,0.0,0.0,0.0,87.0,18.0,105.0,68989185.0,0.0,1.00
3,20,A,100004,99999,1.0,9999,12007,01,99,2.0,9999,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00
4,20,A,100005,99999,1.0,9999,12007,01,99,2.0,9999,1.0,1.0,23439.0,0.0,0.0,297.0,0.0,297.0,9372509.0,0.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66848,20,A,223218,99999,2.0,9999,52101,36,99,1.0,9999,1.0,1.0,105000.0,0.0,5000.0,366.0,0.0,366.0,12183952.0,0.0,9.25
66849,20,A,223219,99999,2.0,9999,52101,36,99,1.0,9999,1.0,1.0,0.0,0.0,0.0,366.0,0.0,366.0,9961082.0,0.0,9.25
66850,20,A,223220,99999,2.0,9999,52101,36,99,1.0,9999,1.0,1.0,1287200.0,633636.0,1355914.0,366.0,0.0,366.0,541328560.0,0.0,9.25
66851,20,A,223221,99999,2.0,9999,58111,36,99,2.0,9999,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.00


Dropping all columns beyond a12 because I cant find the necessary questions in the Block A schedule to which these values correspond to.


In [30]:
blkA = blkA.iloc[:, 0:13].drop(["blk", "yr"], axis=1)
blkA

,a1,a2,a3,a4,a5,a7,a8,a9,a10,a11,a12
0,100001,99999,1.0,9999,12007,01,99,2.0,9999,1.0,2.0
1,100002,99999,1.0,9999,12007,01,99,2.0,9999,1.0,1.0
2,100003,99999,1.0,9999,12007,01,99,1.0,9999,1.0,1.0
3,100004,99999,1.0,9999,12007,01,99,2.0,9999,1.0,7.0
4,100005,99999,1.0,9999,12007,01,99,2.0,9999,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
66848,223218,99999,2.0,9999,52101,36,99,1.0,9999,1.0,1.0
66849,223219,99999,2.0,9999,52101,36,99,1.0,9999,1.0,1.0
66850,223220,99999,2.0,9999,52101,36,99,1.0,9999,1.0,1.0
66851,223221,99999,2.0,9999,58111,36,99,2.0,9999,1.0,4.0


In [31]:
blkA.rename(
    columns={
        "a1": "dsl",
        "a2": "psl",
        "a3": "scheme_code",
        "a4": "nic_4",
        "a5": "nic_5",
        "a7": "state_code",
        "a8": "district_code",
        "a9": "sector",
        "a10": "ro_sro_code",
        "a11": "num_of_units",
        "a12": "unit_status",
    },
    inplace=True,
)
blkA

,dsl,psl,scheme_code,nic_4,nic_5,state_code,district_code,sector,ro_sro_code,num_of_units,unit_status
0,100001,99999,1.0,9999,12007,01,99,2.0,9999,1.0,2.0
1,100002,99999,1.0,9999,12007,01,99,2.0,9999,1.0,1.0
2,100003,99999,1.0,9999,12007,01,99,1.0,9999,1.0,1.0
3,100004,99999,1.0,9999,12007,01,99,2.0,9999,1.0,7.0
4,100005,99999,1.0,9999,12007,01,99,2.0,9999,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
66848,223218,99999,2.0,9999,52101,36,99,1.0,9999,1.0,1.0
66849,223219,99999,2.0,9999,52101,36,99,1.0,9999,1.0,1.0
66850,223220,99999,2.0,9999,52101,36,99,1.0,9999,1.0,1.0
66851,223221,99999,2.0,9999,58111,36,99,2.0,9999,1.0,4.0


In [32]:
blkA.drop(['ro_sro_code', 'district_code', 'scheme_code', 'nic_4', 'psl'], axis=1, inplace=True)
blkA

,dsl,nic_5,state_code,sector,num_of_units,unit_status
0,100001,12007,01,2.0,1.0,2.0
1,100002,12007,01,2.0,1.0,1.0
2,100003,12007,01,1.0,1.0,1.0
3,100004,12007,01,2.0,1.0,7.0
4,100005,12007,01,2.0,1.0,1.0
...,...,...,...,...,...,...
66848,223218,52101,36,1.0,1.0,1.0
66849,223219,52101,36,1.0,1.0,1.0
66850,223220,52101,36,1.0,1.0,1.0
66851,223221,58111,36,2.0,1.0,4.0


In [33]:
blkA['dsl'].nunique()

66853

In [34]:
blkJ[blkJ['dsl']=='119769']

,dsl,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value
41390,119769,1.0,1730002,Cubic Meter,4090276.00,4090276.00,6.717532e+09,762970780.0,0.0,36789882.0,0.0,1446.79,5.917771e+09
41391,119769,2.0,1710008,Megawatt,976809.00,976809.00,2.784574e+09,164103820.0,0.0,21065118.0,0.0,2661.12,2.599405e+09
41392,119769,3.0,1800001,Th. K. Litre,208447.75,208447.75,6.367861e+07,9565574.0,0.0,0.0,0.0,259.60,5.411304e+07


Its was noticed that one firm with DSL = 119769 was a thermal power plant producing 976809 megawatts of electricty. However, this plant was provided an NIC 5 code = 35301. This NIC code refers to firms that produce steam and hot water. Instead this firm must come under NIC code 35102 which corresponts to units producing energy from thermal power using coal. So the firm's NIC is replaced to 35102

In [35]:
blkA['nic_5'] = np.where(blkA['dsl']=='119769', '35102', blkA['nic_5'])

blkA[blkA['dsl']=='119769']

,dsl,nic_5,state_code,sector,num_of_units,unit_status
17975,119769,35102,20,2.0,1.0,1.0


### Merging Block A with WaterJH

In [36]:
waterJHA = pd.merge(blkA, waterJH, on='dsl', how='right', validate="1:m")
waterJHA

,dsl,nic_5,state_code,sector,num_of_units,unit_status,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit,prod_count,multi_prod_indic
0,100006,35101,01,1.0,1.0,1.0,1.0,1710003,Megawatt,1091170.64,1091170.64,3.769026e+09,0.0,0.0,4749436.0,0.0,3449.76,3.764277e+09,1.0,Natural water nec,Th. K. Litre,3.974902e+09,118094328.0,29.71,1,0
1,100016,10304,01,2.0,1.0,1.0,1.0,2143999,Litres,3662469.99,3587556.00,1.173326e+08,0.0,0.0,6118329.0,0.0,31.00,1.135366e+08,4.0,Water for industrial use,Th. K. Litre,3.282010e+06,2545623.0,775.63,1,0
2,100078,11011,01,2.0,1.0,1.0,1.0,2413101,Litres,592912.99,595394.00,3.215130e+07,0.0,0.0,3321700.0,0.0,48.42,2.870947e+07,4.0,Natural water nec,Th. K. Litre,1.163322e+07,345623.0,29.71,1,0
3,100085,11045,01,1.0,1.0,1.0,4.0,2449002,Litres,5252280.00,5252280.00,1.050456e+08,0.0,569032.0,1638400.0,0.0,19.58,1.028382e+08,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,3,1
4,100085,11045,01,1.0,1.0,1.0,5.0,2441003,Litres,2458109.97,2350600.00,2.949493e+07,0.0,0.0,910983.0,0.0,12.16,2.989130e+07,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,221805,10505,36,1.0,1.0,1.0,1.0,2227003,Kg,5426535.00,5330085.00,8.829286e+08,25716366.0,0.0,14449166.0,0.0,158.11,8.580132e+08,5.0,Water for industrial use,Th. K. Litre,4.268996e+08,100680000.0,235.84,1,0
675,222173,11049,36,2.0,1.0,1.0,1.0,2441099,Litres,203059.81,203059.81,1.460000e+06,0.0,0.0,0.0,0.0,7.19,1.460000e+06,1.0,Natural water nec,Th. K. Litre,8.246380e+06,245000.0,29.71,1,0
676,222175,11043,36,1.0,1.0,1.0,1.0,2441001,Litres,1864483.01,1678516.00,2.402531e+07,0.0,0.0,274298.0,0.0,14.15,2.638244e+07,1.0,Natural water nec,Th. K. Litre,4.469379e+08,13278524.0,29.71,1,0
677,222176,11049,36,1.0,1.0,1.0,1.0,2441099,Litres,122461.75,122461.75,8.805000e+05,0.0,0.0,0.0,0.0,7.19,8.805000e+05,1.0,Natural water nec,Th. K. Litre,8.799700e+05,100000.0,113.64,1,0


### Merging WaterJHA to NIC codes


In [37]:
nic_df = pd.read_parquet(nic_path).drop("s_no", axis=1)

nic_cols = ["nic_" + x for x in nic_df.columns]

nic_df.columns = nic_cols

nic_df

,nic_description,nic_sub_class,nic_class,nic_group,nic_division,nic_section
0,Growing of wheat,01111,0111,011,"Crop and animal production, hunting and relate...","Agriculture, forestry and fishing"
1,"Growing of jowar, bajra and millets",01112,0111,011,"Crop and animal production, hunting and relate...","Agriculture, forestry and fishing"
2,Growing of other cereals,01113,0111,011,"Crop and animal production, hunting and relate...","Agriculture, forestry and fishing"
3,Growing of pulses (dal) and other leguminous c...,01114,0111,011,"Crop and animal production, hunting and relate...","Agriculture, forestry and fishing"
4,Growing of mustard oil seed,01115,0111,011,"Crop and animal production, hunting and relate...","Agriculture, forestry and fishing"
...,...,...,...,...,...,...
1292,General household maintenance activities like ...,96908,9690,969,Other personal service activities,Other service activities
1293,Activities of households as employers of domes...,97000,9700,970,Activities of households as employers of domes...,Activities of households as employers; undiffe...
1294,Undifferentiated goods-producing activities of...,98100,9810,981,Undifferentiated goods- and services-producing...,Activities of households as employers; undiffe...
1295,Undifferentiated service-producing activities ...,98200,9820,982,Undifferentiated goods- and services-producing...,Activities of households as employers; undiffe...


In [38]:
waterJHA_nic = pd.merge(
    waterJHA, nic_df, left_on="nic_5", right_on="nic_sub_class", how="left", validate="m:1"
)
waterJHA_nic.drop(["nic_5"], axis=1, inplace=True)

waterJHA_nic

,dsl,state_code,sector,num_of_units,unit_status,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit,prod_count,multi_prod_indic,nic_description,nic_sub_class,nic_class,nic_group,nic_division,nic_section
0,100006,01,1.0,1.0,1.0,1.0,1710003,Megawatt,1091170.64,1091170.64,3.769026e+09,0.0,0.0,4749436.0,0.0,3449.76,3.764277e+09,1.0,Natural water nec,Th. K. Litre,3.974902e+09,118094328.0,29.71,1,0,Electric power generation by hydroelectric pow...,35101,3510,351,"Electricity, gas, steam and air conditioning s...","Electricity, gas, steam and air conditioning s..."
1,100016,01,2.0,1.0,1.0,1.0,2143999,Litres,3662469.99,3587556.00,1.173326e+08,0.0,0.0,6118329.0,0.0,31.00,1.135366e+08,4.0,Water for industrial use,Th. K. Litre,3.282010e+06,2545623.0,775.63,1,0,Manufacture of fruit or vegetable juices and t...,10304,1030,103,Manufacture of food products,Manufacturing
2,100078,01,2.0,1.0,1.0,1.0,2413101,Litres,592912.99,595394.00,3.215130e+07,0.0,0.0,3321700.0,0.0,48.42,2.870947e+07,4.0,Natural water nec,Th. K. Litre,1.163322e+07,345623.0,29.71,1,0,"Manufacture of distilled, potable, alcoholic b...",11011,1101,110,Manufacture of beverages,Manufacturing
3,100085,01,1.0,1.0,1.0,4.0,2449002,Litres,5252280.00,5252280.00,1.050456e+08,0.0,569032.0,1638400.0,0.0,19.58,1.028382e+08,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,3,1,Manufacture of soft drinks,11045,1104,110,Manufacture of beverages,Manufacturing
4,100085,01,1.0,1.0,1.0,5.0,2441003,Litres,2458109.97,2350600.00,2.949493e+07,0.0,0.0,910983.0,0.0,12.16,2.989130e+07,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,3,1,Manufacture of soft drinks,11045,1104,110,Manufacture of beverages,Manufacturing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,221805,36,1.0,1.0,1.0,1.0,2227003,Kg,5426535.00,5330085.00,8.829286e+08,25716366.0,0.0,14449166.0,0.0,158.11,8.580132e+08,5.0,Water for industrial use,Th. K. Litre,4.268996e+08,100680000.0,235.84,1,0,"Manufacture of ice-cream, kulfi etc.",10505,1050,105,Manufacture of food products,Manufacturing
675,222173,36,2.0,1.0,1.0,1.0,2441099,Litres,203059.81,203059.81,1.460000e+06,0.0,0.0,0.0,0.0,7.19,1.460000e+06,1.0,Natural water nec,Th. K. Litre,8.246380e+06,245000.0,29.71,1,0,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing
676,222175,36,1.0,1.0,1.0,1.0,2441001,Litres,1864483.01,1678516.00,2.402531e+07,0.0,0.0,274298.0,0.0,14.15,2.638244e+07,1.0,Natural water nec,Th. K. Litre,4.469379e+08,13278524.0,29.71,1,0,Manufacture of mineral water,11043,1104,110,Manufacture of beverages,Manufacturing
677,222176,36,1.0,1.0,1.0,1.0,2441099,Litres,122461.75,122461.75,8.805000e+05,0.0,0.0,0.0,0.0,7.19,8.805000e+05,1.0,Natural water nec,Th. K. Litre,8.799700e+05,100000.0,113.64,1,0,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing


### Merging WaterJHA_nic to NPCMS

In [39]:
npcms = pd.read_excel(npcms_path,dtype={"npcms":str})

npcms_cols = ["npcms_" + x for x in npcms.columns]

npcms.columns = npcms_cols

npcms

,npcms_npcms,npcms_desc_item,npcms_unit
0,0111100,"Wheat, seed",Tonne
1,0111200,"Wheat, other",Tonne
2,0112100,"Maize (corn), seed",Tonne
3,0112200,"Maize (corn), other",Tonne
4,0113100,"Rice, seed",Tonne
...,...,...,...
6214,4994203,Leg-guards / mud-guard,No
6215,4994204,"Nipples, cycle",No
6216,4994205,Rubber brake lining,No
6217,4994206,"Spokes, iron / steel",No


In [40]:
waterJHA_nic_npcms = pd.merge(waterJHA_nic, npcms, left_on='output_item_code', right_on='npcms_npcms', how='left', validate='m:1').drop(['npcms_npcms','prod_count'], axis=1)

waterJHA_nic_npcms

,dsl,state_code,sector,num_of_units,unit_status,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit,multi_prod_indic,nic_description,nic_sub_class,nic_class,nic_group,nic_division,nic_section,npcms_desc_item,npcms_unit
0,100006,01,1.0,1.0,1.0,1.0,1710003,Megawatt,1091170.64,1091170.64,3.769026e+09,0.0,0.0,4749436.0,0.0,3449.76,3.764277e+09,1.0,Natural water nec,Th. K. Litre,3.974902e+09,118094328.0,29.71,0,Electric power generation by hydroelectric pow...,35101,3510,351,"Electricity, gas, steam and air conditioning s...","Electricity, gas, steam and air conditioning s...",Electricity from hydro energy,Megawatt
1,100016,01,2.0,1.0,1.0,1.0,2143999,Litres,3662469.99,3587556.00,1.173326e+08,0.0,0.0,6118329.0,0.0,31.00,1.135366e+08,4.0,Water for industrial use,Th. K. Litre,3.282010e+06,2545623.0,775.63,0,Manufacture of fruit or vegetable juices and t...,10304,1030,103,Manufacture of food products,Manufacturing,"Other fruit juices, n.e.c.",Litre
2,100078,01,2.0,1.0,1.0,1.0,2413101,Litres,592912.99,595394.00,3.215130e+07,0.0,0.0,3321700.0,0.0,48.42,2.870947e+07,4.0,Natural water nec,Th. K. Litre,1.163322e+07,345623.0,29.71,0,"Manufacture of distilled, potable, alcoholic b...",11011,1101,110,Manufacture of beverages,Manufacturing,Country liquor and its variants,Litre
3,100085,01,1.0,1.0,1.0,4.0,2449002,Litres,5252280.00,5252280.00,1.050456e+08,0.0,569032.0,1638400.0,0.0,19.58,1.028382e+08,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,1,Manufacture of soft drinks,11045,1104,110,Manufacture of beverages,Manufacturing,"Soft drink , carbonated beverage other than so...",Litre
4,100085,01,1.0,1.0,1.0,5.0,2441003,Litres,2458109.97,2350600.00,2.949493e+07,0.0,0.0,910983.0,0.0,12.16,2.989130e+07,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,1,Manufacture of soft drinks,11045,1104,110,Manufacture of beverages,Manufacturing,Packed drinking water,Litre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,221805,36,1.0,1.0,1.0,1.0,2227003,Kg,5426535.00,5330085.00,8.829286e+08,25716366.0,0.0,14449166.0,0.0,158.11,8.580132e+08,5.0,Water for industrial use,Th. K. Litre,4.268996e+08,100680000.0,235.84,0,"Manufacture of ice-cream, kulfi etc.",10505,1050,105,Manufacture of food products,Manufacturing,Ice cream,Kg
675,222173,36,2.0,1.0,1.0,1.0,2441099,Litres,203059.81,203059.81,1.460000e+06,0.0,0.0,0.0,0.0,7.19,1.460000e+06,1.0,Natural water nec,Th. K. Litre,8.246380e+06,245000.0,29.71,0,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing,"Bottled waters, not sweetened or flavoured, n....",Litre
676,222175,36,1.0,1.0,1.0,1.0,2441001,Litres,1864483.01,1678516.00,2.402531e+07,0.0,0.0,274298.0,0.0,14.15,2.638244e+07,1.0,Natural water nec,Th. K. Litre,4.469379e+08,13278524.0,29.71,0,Manufacture of mineral water,11043,1104,110,Manufacture of beverages,Manufacturing,Bottled Mineral Water,Litre
677,222176,36,1.0,1.0,1.0,1.0,2441099,Litres,122461.75,122461.75,8.805000e+05,0.0,0.0,0.0,0.0,7.19,8.805000e+05,1.0,Natural water nec,Th. K. Litre,8.799700e+05,100000.0,113.64,0,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing,"Bottled waters, not sweetened or flavoured, n....",Litre


### Isolating units with more than 1 output

In [41]:
cols_to_isolate = ['dsl', "nic_description",	"nic_sub_class", "nic_class", "nic_group", "nic_division", "nic_section", 'output_sl_no', 'output_item_code', 'output_unit_qty_code', 'output_qty_manuf', 'npcms_desc_item', 'npcms_unit', "input_sl_no","input_npcms_item_code","input_unit_qty_code","input_qty_consumed","input_purchase_value","input_rate_per_unit"]

multi_prod_units = waterJHA_nic_npcms[waterJHA_nic_npcms['multi_prod_indic']==1][cols_to_isolate]

multi_prod_units

,dsl,nic_description,nic_sub_class,nic_class,nic_group,nic_division,nic_section,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,npcms_desc_item,npcms_unit,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit
3,100085,Manufacture of soft drinks,11045,1104,110,Manufacture of beverages,Manufacturing,4.0,2449002,Litres,5252280.00,"Soft drink , carbonated beverage other than so...",Litre,5.0,Natural water nec,Th. K. Litre,62470550.0,1856000.0,29.71
4,100085,Manufacture of soft drinks,11045,1104,110,Manufacture of beverages,Manufacturing,5.0,2441003,Litres,2458109.97,Packed drinking water,Litre,5.0,Natural water nec,Th. K. Litre,62470550.0,1856000.0,29.71
5,100085,Manufacture of soft drinks,11045,1104,110,Manufacture of beverages,Manufacturing,6.0,2449001,Litres,369438.84,Soda water,Litre,5.0,Natural water nec,Th. K. Litre,62470550.0,1856000.0,29.71
6,100087,Manufacture of aerated drinks,11041,1104,110,Manufacture of beverages,Manufacturing,1.0,2449002,Litres,637271.27,"Soft drink , carbonated beverage other than so...",Litre,5.0,Water for domestic use,Th. K. Litre,80930.0,50000.0,617.84
7,100087,Manufacture of aerated drinks,11041,1104,110,Manufacture of beverages,Manufacturing,2.0,2441099,Litres,1128700.00,"Bottled waters, not sweetened or flavoured, n....",Litre,5.0,Water for domestic use,Th. K. Litre,80930.0,50000.0,617.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,211830,Manufacture of synthetic or artificial filamen...,20302,2030,203,Manufacture of chemicals and chemical products,Manufacturing,3.0,3712105,Kg,843349.00,Fibre glass yarn,Kg,5.0,Water for industrial use,Th. K. Litre,1719010.0,405411.0,235.84
649,216422,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing,2.0,2449002,Litres,435495.99,"Soft drink , carbonated beverage other than so...",Litre,1.0,Natural water nec,Th. K. Litre,3567820.0,106000.0,29.71
650,216422,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing,3.0,2441003,Litres,978182.98,Packed drinking water,Litre,1.0,Natural water nec,Th. K. Litre,3567820.0,106000.0,29.71
654,217674,Manufacture of mineral water,11043,1104,110,Manufacture of beverages,Manufacturing,1.0,2441003,Litres,332625.20,Packed drinking water,Litre,1.0,Natural water nec,Th. K. Litre,1039650.0,30888.0,29.71


In [42]:
one_prod_units = waterJHA_nic_npcms[waterJHA_nic_npcms['multi_prod_indic']==0][cols_to_isolate]

one_prod_units

,dsl,nic_description,nic_sub_class,nic_class,nic_group,nic_division,nic_section,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,npcms_desc_item,npcms_unit,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit
0,100006,Electric power generation by hydroelectric pow...,35101,3510,351,"Electricity, gas, steam and air conditioning s...","Electricity, gas, steam and air conditioning s...",1.0,1710003,Megawatt,1091170.64,Electricity from hydro energy,Megawatt,1.0,Natural water nec,Th. K. Litre,3.974902e+09,118094328.0,29.71
1,100016,Manufacture of fruit or vegetable juices and t...,10304,1030,103,Manufacture of food products,Manufacturing,1.0,2143999,Litres,3662469.99,"Other fruit juices, n.e.c.",Litre,4.0,Water for industrial use,Th. K. Litre,3.282010e+06,2545623.0,775.63
2,100078,"Manufacture of distilled, potable, alcoholic b...",11011,1101,110,Manufacture of beverages,Manufacturing,1.0,2413101,Litres,592912.99,Country liquor and its variants,Litre,4.0,Natural water nec,Th. K. Litre,1.163322e+07,345623.0,29.71
8,100089,Manufacture of mineral water,11043,1104,110,Manufacture of beverages,Manufacturing,1.0,2441099,Litres,424244.99,"Bottled waters, not sweetened or flavoured, n....",Litre,3.0,Water for industrial use,Th. K. Litre,3.873000e+04,9135.0,235.84
11,100092,Manufacture of mineral water,11043,1104,110,Manufacture of beverages,Manufacturing,1.0,2441099,Litres,280201.98,"Bottled waters, not sweetened or flavoured, n....",Litre,7.0,Water for industrial use,Th. K. Litre,1.077800e+05,25420.0,235.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,221805,"Manufacture of ice-cream, kulfi etc.",10505,1050,105,Manufacture of food products,Manufacturing,1.0,2227003,Kg,5426535.00,Ice cream,Kg,5.0,Water for industrial use,Th. K. Litre,4.268996e+08,100680000.0,235.84
675,222173,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing,1.0,2441099,Litres,203059.81,"Bottled waters, not sweetened or flavoured, n....",Litre,1.0,Natural water nec,Th. K. Litre,8.246380e+06,245000.0,29.71
676,222175,Manufacture of mineral water,11043,1104,110,Manufacture of beverages,Manufacturing,1.0,2441001,Litres,1864483.01,Bottled Mineral Water,Litre,1.0,Natural water nec,Th. K. Litre,4.469379e+08,13278524.0,29.71
677,222176,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing,1.0,2441099,Litres,122461.75,"Bottled waters, not sweetened or flavoured, n....",Litre,1.0,Natural water nec,Th. K. Litre,8.799700e+05,100000.0,113.64


In [43]:
# multi_prod_units.to_csv(data_folder.joinpath("multi_prod_units.csv"), index=False)
# one_prod_units.to_csv(data_folder.joinpath("one_prod_units.csv"), index=False)

### Marking the valid outputs 

Using the multi_prod_units dataframe, we manually inspected all the 301 entries and marked main products and by products using a binary column of the name "valid_outputs".

If Valid_outputs = 1, then the product is in line with the NIC description and should be retained. We identify these outputs using their output_sl_no column value

We undertake this effort so that we can measure quantity of water used for production of only main products and the quanitty of by products will not get inserted in this calculation

In [44]:
valid_outs = pd.read_csv(valid_output_marker_path, dtype={'dsl':str})
valid_outs = valid_outs[valid_outs['valid_outputs']==1][["dsl", "output_sl_no"]]
valid_outs = pd.concat([one_prod_units[["dsl", "output_sl_no"]], valid_outs], axis=0)
valid_outs

,dsl,output_sl_no
0,100006,1.0
1,100016,1.0
2,100078,1.0
8,100089,1.0
11,100092,1.0
...,...,...
296,211830,3.0
297,216422,2.0
298,216422,3.0
299,217674,1.0


Retaining only the valid outputs

In [45]:
asi_valid_outputs = pd.merge(waterJHA_nic_npcms, valid_outs, on=['dsl','output_sl_no'], how='right', validate='1:1')
asi_valid_outputs

,dsl,state_code,sector,num_of_units,unit_status,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit,multi_prod_indic,nic_description,nic_sub_class,nic_class,nic_group,nic_division,nic_section,npcms_desc_item,npcms_unit
0,100006,01,1.0,1.0,1.0,1.0,1710003,Megawatt,1091170.64,1091170.64,3.769026e+09,0.0,0.0,4749436.0,0.0,3449.76,3.764277e+09,1.0,Natural water nec,Th. K. Litre,3.974902e+09,118094328.0,29.71,0,Electric power generation by hydroelectric pow...,35101,3510,351,"Electricity, gas, steam and air conditioning s...","Electricity, gas, steam and air conditioning s...",Electricity from hydro energy,Megawatt
1,100016,01,2.0,1.0,1.0,1.0,2143999,Litres,3662469.99,3587556.00,1.173326e+08,0.0,0.0,6118329.0,0.0,31.00,1.135366e+08,4.0,Water for industrial use,Th. K. Litre,3.282010e+06,2545623.0,775.63,0,Manufacture of fruit or vegetable juices and t...,10304,1030,103,Manufacture of food products,Manufacturing,"Other fruit juices, n.e.c.",Litre
2,100078,01,2.0,1.0,1.0,1.0,2413101,Litres,592912.99,595394.00,3.215130e+07,0.0,0.0,3321700.0,0.0,48.42,2.870947e+07,4.0,Natural water nec,Th. K. Litre,1.163322e+07,345623.0,29.71,0,"Manufacture of distilled, potable, alcoholic b...",11011,1101,110,Manufacture of beverages,Manufacturing,Country liquor and its variants,Litre
3,100089,01,2.0,1.0,1.0,1.0,2441099,Litres,424244.99,385708.00,6.141486e+06,934424.0,0.0,61311.0,0.0,13.34,5.659875e+06,3.0,Water for industrial use,Th. K. Litre,3.873000e+04,9135.0,235.84,0,Manufacture of mineral water,11043,1104,110,Manufacture of beverages,Manufacturing,"Bottled waters, not sweetened or flavoured, n....",Litre
4,100092,01,2.0,1.0,1.0,1.0,2441099,Litres,280201.98,371645.00,5.035796e+06,559532.0,0.0,0.0,0.0,12.04,3.374882e+06,7.0,Water for industrial use,Th. K. Litre,1.077800e+05,25420.0,235.84,0,Manufacture of mineral water,11043,1104,110,Manufacture of beverages,Manufacturing,"Bottled waters, not sweetened or flavoured, n....",Litre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,211830,25,1.0,1.0,1.0,3.0,3712105,Kg,843349.00,848797.00,9.099116e+07,0.0,0.0,3089670.0,0.0,103.56,8.733729e+07,5.0,Water for industrial use,Th. K. Litre,1.719010e+06,405411.0,235.84,1,Manufacture of synthetic or artificial filamen...,20302,2030,203,Manufacture of chemicals and chemical products,Manufacturing,Fibre glass yarn,Kg
629,216422,29,2.0,1.0,1.0,2.0,2449002,Litres,435495.99,317369.00,9.521084e+06,0.0,0.0,0.0,0.0,30.00,1.306490e+07,1.0,Natural water nec,Th. K. Litre,3.567820e+06,106000.0,29.71,1,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing,"Soft drink , carbonated beverage other than so...",Litre
630,216422,29,2.0,1.0,1.0,3.0,2441003,Litres,978182.98,897416.00,8.076746e+06,0.0,0.0,0.0,0.0,9.00,8.803649e+06,1.0,Natural water nec,Th. K. Litre,3.567820e+06,106000.0,29.71,1,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing,Packed drinking water,Litre
631,217674,32,2.0,1.0,1.0,1.0,2441003,Litres,332625.20,282894.00,5.445920e+05,0.0,0.0,1435.0,0.0,1.92,6.386410e+05,1.0,Natural water nec,Th. K. Litre,1.039650e+06,30888.0,29.71,1,Manufacture of mineral water,11043,1104,110,Manufacture of beverages,Manufacturing,Packed drinking water,Litre


## Output Standardization

Now that we have isolated the valid outputs for all the industrial units, we must ensure that all these outputs are measured in the same metric so that calulation of total output by each industrial unit within each NIC code within each state is possible. 

In [46]:
asi_out_units = asi_valid_outputs[['dsl', "nic_description",	"nic_sub_class", 'output_sl_no', 'output_item_code', 'output_unit_qty_code', 'output_qty_manuf', 'npcms_desc_item', 'npcms_unit']]
# asi_out_units.to_csv(data_folder.joinpath('asi_output_units.csv'), index=False)
asi_out_units


,dsl,nic_description,nic_sub_class,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,npcms_desc_item,npcms_unit
0,100006,Electric power generation by hydroelectric pow...,35101,1.0,1710003,Megawatt,1091170.64,Electricity from hydro energy,Megawatt
1,100016,Manufacture of fruit or vegetable juices and t...,10304,1.0,2143999,Litres,3662469.99,"Other fruit juices, n.e.c.",Litre
2,100078,"Manufacture of distilled, potable, alcoholic b...",11011,1.0,2413101,Litres,592912.99,Country liquor and its variants,Litre
3,100089,Manufacture of mineral water,11043,1.0,2441099,Litres,424244.99,"Bottled waters, not sweetened or flavoured, n....",Litre
4,100092,Manufacture of mineral water,11043,1.0,2441099,Litres,280201.98,"Bottled waters, not sweetened or flavoured, n....",Litre
...,...,...,...,...,...,...,...,...,...
628,211830,Manufacture of synthetic or artificial filamen...,20302,3.0,3712105,Kg,843349.00,Fibre glass yarn,Kg
629,216422,Manufacture of other non-alcoholic beverages n...,11049,2.0,2449002,Litres,435495.99,"Soft drink , carbonated beverage other than so...",Litre
630,216422,Manufacture of other non-alcoholic beverages n...,11049,3.0,2441003,Litres,978182.98,Packed drinking water,Litre
631,217674,Manufacture of mineral water,11043,1.0,2441003,Litres,332625.20,Packed drinking water,Litre


### Bring in updated output conversion factors and updating output manufactured
To estimate water demnand for each state and each NIC code, we need to ensure that the metric of production is uniform across industrial units within each NIC code. This was achieved manually by correcting the "./data/asi/external/asi_output_units.xlsx". 

In [47]:
metric_convert_df = pd.read_excel(metric_converter_path, dtype={'dsl':str, 'nic_sub_class':str})[['dsl','nic_sub_class', 'output_sl_no', 'metric_conversion_factor','output_standard_metric']]
metric_convert_df

,dsl,nic_sub_class,output_sl_no,metric_conversion_factor,output_standard_metric
0,100006,35101,1,1.000,Megawatt
1,100016,10304,1,1.000,Litres
2,100078,11011,1,1.000,Litres
3,100089,11043,1,1.000,Litres
4,100092,11043,1,1.000,Litres
...,...,...,...,...,...
628,211830,20302,3,0.001,Tonne
629,216422,11049,2,1.000,Litres
630,216422,11049,3,1.000,Litres
631,217674,11043,1,1.000,Litres


Merging metric conversion frame to the water data frame

In [48]:
asi_final = pd.merge(asi_valid_outputs, metric_convert_df, on=['dsl','nic_sub_class', 'output_sl_no'], how='outer', validate="1:1")
asi_final

,dsl,state_code,sector,num_of_units,unit_status,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit,multi_prod_indic,nic_description,nic_sub_class,nic_class,nic_group,nic_division,nic_section,npcms_desc_item,npcms_unit,metric_conversion_factor,output_standard_metric
0,100006,01,1.0,1.0,1.0,1.0,1710003,Megawatt,1091170.64,1091170.64,3.769026e+09,0.0,0.0,4749436.0,0.0,3449.76,3.764277e+09,1.0,Natural water nec,Th. K. Litre,3.974902e+09,118094328.0,29.71,0,Electric power generation by hydroelectric pow...,35101,3510,351,"Electricity, gas, steam and air conditioning s...","Electricity, gas, steam and air conditioning s...",Electricity from hydro energy,Megawatt,1.000,Megawatt
1,100016,01,2.0,1.0,1.0,1.0,2143999,Litres,3662469.99,3587556.00,1.173326e+08,0.0,0.0,6118329.0,0.0,31.00,1.135366e+08,4.0,Water for industrial use,Th. K. Litre,3.282010e+06,2545623.0,775.63,0,Manufacture of fruit or vegetable juices and t...,10304,1030,103,Manufacture of food products,Manufacturing,"Other fruit juices, n.e.c.",Litre,1.000,Litres
2,100078,01,2.0,1.0,1.0,1.0,2413101,Litres,592912.99,595394.00,3.215130e+07,0.0,0.0,3321700.0,0.0,48.42,2.870947e+07,4.0,Natural water nec,Th. K. Litre,1.163322e+07,345623.0,29.71,0,"Manufacture of distilled, potable, alcoholic b...",11011,1101,110,Manufacture of beverages,Manufacturing,Country liquor and its variants,Litre,1.000,Litres
3,100085,01,1.0,1.0,1.0,4.0,2449002,Litres,5252280.00,5252280.00,1.050456e+08,0.0,569032.0,1638400.0,0.0,19.58,1.028382e+08,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,1,Manufacture of soft drinks,11045,1104,110,Manufacture of beverages,Manufacturing,"Soft drink , carbonated beverage other than so...",Litre,1.000,Litres
4,100085,01,1.0,1.0,1.0,5.0,2441003,Litres,2458109.97,2350600.00,2.949493e+07,0.0,0.0,910983.0,0.0,12.16,2.989130e+07,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,1,Manufacture of soft drinks,11045,1104,110,Manufacture of beverages,Manufacturing,Packed drinking water,Litre,1.000,Litres
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,221805,36,1.0,1.0,1.0,1.0,2227003,Kg,5426535.00,5330085.00,8.829286e+08,25716366.0,0.0,14449166.0,0.0,158.11,8.580132e+08,5.0,Water for industrial use,Th. K. Litre,4.268996e+08,100680000.0,235.84,0,"Manufacture of ice-cream, kulfi etc.",10505,1050,105,Manufacture of food products,Manufacturing,Ice cream,Kg,0.001,Tonne
629,222173,36,2.0,1.0,1.0,1.0,2441099,Litres,203059.81,203059.81,1.460000e+06,0.0,0.0,0.0,0.0,7.19,1.460000e+06,1.0,Natural water nec,Th. K. Litre,8.246380e+06,245000.0,29.71,0,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing,"Bottled waters, not sweetened or flavoured, n....",Litre,1.000,Litres
630,222175,36,1.0,1.0,1.0,1.0,2441001,Litres,1864483.01,1678516.00,2.402531e+07,0.0,0.0,274298.0,0.0,14.15,2.638244e+07,1.0,Natural water nec,Th. K. Litre,4.469379e+08,13278524.0,29.71,0,Manufacture of mineral water,11043,1104,110,Manufacture of beverages,Manufacturing,Bottled Mineral Water,Litre,1.000,Litres
631,222176,36,1.0,1.0,1.0,1.0,2441099,Litres,122461.75,122461.75,8.805000e+05,0.0,0.0,0.0,0.0,7.19,8.805000e+05,1.0,Natural water nec,Th. K. Litre,8.799700e+05,100000.0,113.64,0,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing,"Bottled waters, not sweetened or flavoured, n....",Litre,1.000,Litres


In [49]:
asi_final['output_manuf_metric_converted'] = asi_final['metric_conversion_factor'] * asi_final['output_qty_manuf']
asi_final

,dsl,state_code,sector,num_of_units,unit_status,output_sl_no,output_item_code,output_unit_qty_code,output_qty_manuf,output_qty_sold,gross_sales_value,gst,other_tax,other_distributive_exp,subsidy,per_unit_net_sales_value,ex_factory_value,input_sl_no,input_npcms_item_code,input_unit_qty_code,input_qty_consumed,input_purchase_value,input_rate_per_unit,multi_prod_indic,nic_description,nic_sub_class,nic_class,nic_group,nic_division,nic_section,npcms_desc_item,npcms_unit,metric_conversion_factor,output_standard_metric,output_manuf_metric_converted
0,100006,01,1.0,1.0,1.0,1.0,1710003,Megawatt,1091170.64,1091170.64,3.769026e+09,0.0,0.0,4749436.0,0.0,3449.76,3.764277e+09,1.0,Natural water nec,Th. K. Litre,3.974902e+09,118094328.0,29.71,0,Electric power generation by hydroelectric pow...,35101,3510,351,"Electricity, gas, steam and air conditioning s...","Electricity, gas, steam and air conditioning s...",Electricity from hydro energy,Megawatt,1.000,Megawatt,1091170.640
1,100016,01,2.0,1.0,1.0,1.0,2143999,Litres,3662469.99,3587556.00,1.173326e+08,0.0,0.0,6118329.0,0.0,31.00,1.135366e+08,4.0,Water for industrial use,Th. K. Litre,3.282010e+06,2545623.0,775.63,0,Manufacture of fruit or vegetable juices and t...,10304,1030,103,Manufacture of food products,Manufacturing,"Other fruit juices, n.e.c.",Litre,1.000,Litres,3662469.990
2,100078,01,2.0,1.0,1.0,1.0,2413101,Litres,592912.99,595394.00,3.215130e+07,0.0,0.0,3321700.0,0.0,48.42,2.870947e+07,4.0,Natural water nec,Th. K. Litre,1.163322e+07,345623.0,29.71,0,"Manufacture of distilled, potable, alcoholic b...",11011,1101,110,Manufacture of beverages,Manufacturing,Country liquor and its variants,Litre,1.000,Litres,592912.990
3,100085,01,1.0,1.0,1.0,4.0,2449002,Litres,5252280.00,5252280.00,1.050456e+08,0.0,569032.0,1638400.0,0.0,19.58,1.028382e+08,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,1,Manufacture of soft drinks,11045,1104,110,Manufacture of beverages,Manufacturing,"Soft drink , carbonated beverage other than so...",Litre,1.000,Litres,5252280.000
4,100085,01,1.0,1.0,1.0,5.0,2441003,Litres,2458109.97,2350600.00,2.949493e+07,0.0,0.0,910983.0,0.0,12.16,2.989130e+07,5.0,Natural water nec,Th. K. Litre,6.247055e+07,1856000.0,29.71,1,Manufacture of soft drinks,11045,1104,110,Manufacture of beverages,Manufacturing,Packed drinking water,Litre,1.000,Litres,2458109.970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,221805,36,1.0,1.0,1.0,1.0,2227003,Kg,5426535.00,5330085.00,8.829286e+08,25716366.0,0.0,14449166.0,0.0,158.11,8.580132e+08,5.0,Water for industrial use,Th. K. Litre,4.268996e+08,100680000.0,235.84,0,"Manufacture of ice-cream, kulfi etc.",10505,1050,105,Manufacture of food products,Manufacturing,Ice cream,Kg,0.001,Tonne,5426.535
629,222173,36,2.0,1.0,1.0,1.0,2441099,Litres,203059.81,203059.81,1.460000e+06,0.0,0.0,0.0,0.0,7.19,1.460000e+06,1.0,Natural water nec,Th. K. Litre,8.246380e+06,245000.0,29.71,0,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing,"Bottled waters, not sweetened or flavoured, n....",Litre,1.000,Litres,203059.810
630,222175,36,1.0,1.0,1.0,1.0,2441001,Litres,1864483.01,1678516.00,2.402531e+07,0.0,0.0,274298.0,0.0,14.15,2.638244e+07,1.0,Natural water nec,Th. K. Litre,4.469379e+08,13278524.0,29.71,0,Manufacture of mineral water,11043,1104,110,Manufacture of beverages,Manufacturing,Bottled Mineral Water,Litre,1.000,Litres,1864483.010
631,222176,36,1.0,1.0,1.0,1.0,2441099,Litres,122461.75,122461.75,8.805000e+05,0.0,0.0,0.0,0.0,7.19,8.805000e+05,1.0,Natural water nec,Th. K. Litre,8.799700e+05,100000.0,113.64,0,Manufacture of other non-alcoholic beverages n...,11049,1104,110,Manufacture of beverages,Manufacturing,"Bottled waters, not sweetened or flavoured, n....",Litre,1.000,Litres,122461.750


In [50]:
asi_final['output_manuf_metric_converted'].isna().value_counts()

output_manuf_metric_converted
False    633
Name: count, dtype: int64

## Estimating Water Footprint per Unit of Production

All states have only one standard output metric for each NIC code

In [51]:
test_df = asi_final.groupby(['state_code', 'nic_sub_class'])['output_standard_metric'].nunique().reset_index()

test_df[test_df['output_standard_metric']>1]

,state_code,nic_sub_class,output_standard_metric


In [52]:
test_df = asi_final.groupby(['state_code', 'nic_sub_class'])['output_standard_metric']\
    .unique()\
    .apply(lambda x: ''.join(map(str, x)))\
    .reset_index()

test_df['metric'] = "cubic meter per " + test_df['output_standard_metric']

test_df.loc[test_df['output_standard_metric']=='nan', ['output_standard_metric', 'metric']] = np.nan

test_df.drop('output_standard_metric', axis=1, inplace=True)

test_df['metric'].isna().value_counts()

metric
False    237
Name: count, dtype: int64

In [53]:
test_df

,state_code,nic_sub_class,metric
0,01,10304,cubic meter per Litres
1,01,11011,cubic meter per Litres
2,01,11041,cubic meter per Litres
3,01,11043,cubic meter per Litres
4,01,11045,cubic meter per Litres
...,...,...,...
232,36,11031,cubic meter per Litres
233,36,11043,cubic meter per Litres
234,36,11044,cubic meter per Tonne
235,36,11049,cubic meter per Litres


Water Footprint Calculation:

The current data frame is at product level, that is every valid output manufactured by the industrial unit and measured in a standardized metric. To estimate water footprint, we must implement the following steps:
1. Take the aggregate of all products manufactured by each industrial unit.
2. Calculate water consumed per unit of production by the firm. 

    water consumed per unit of production = Total water consumed during production by firm / Sum of products of industrial unit

3. Calculate the median water consumed per unit of production across firms within each NIC code and then within each state.

### Step 1 - Sum of outputs per firm
Since outputs have a standardized metric, they can be aggregated

In [54]:
firm_output_sum = asi_final.groupby('dsl')['output_manuf_metric_converted'].sum().reset_index()

firm_output_sum

,dsl,output_manuf_metric_converted
0,100006,1091170.640
1,100016,3662469.990
2,100078,592912.990
3,100085,8079828.810
4,100087,1765971.270
...,...,...
465,221805,5426.535
466,222173,203059.810
467,222175,1864483.010
468,222176,122461.750


In [55]:
water_used = pd.merge(firm_output_sum, blkH_water[['dsl','input_qty_consumed' ]], on='dsl', how='left', validate='1:1')
water_used

,dsl,output_manuf_metric_converted,input_qty_consumed
0,100006,1091170.640,3.974902e+09
1,100016,3662469.990,3.282010e+06
2,100078,592912.990,1.163322e+07
3,100085,8079828.810,6.247055e+07
4,100087,1765971.270,8.093000e+04
...,...,...,...
465,221805,5426.535,4.268996e+08
466,222173,203059.810,8.246380e+06
467,222175,1864483.010,4.469379e+08
468,222176,122461.750,8.799700e+05


### Step 2 - Water consumed per unit of production

In [56]:
water_used['unit_water_consumption']  =  water_used['input_qty_consumed'] / water_used['output_manuf_metric_converted']

water_used

,dsl,output_manuf_metric_converted,input_qty_consumed,unit_water_consumption
0,100006,1091170.640,3.974902e+09,3642.786476
1,100016,3662469.990,3.282010e+06,0.896119
2,100078,592912.990,1.163322e+07,19.620451
3,100085,8079828.810,6.247055e+07,7.731668
4,100087,1765971.270,8.093000e+04,0.045827
...,...,...,...,...
465,221805,5426.535,4.268996e+08,78668.909350
466,222173,203059.810,8.246380e+06,40.610596
467,222175,1864483.010,4.469379e+08,239.711420
468,222176,122461.750,8.799700e+05,7.185672


In [57]:
water_used['unit_water_consumption'].describe().round(2)

count       470.00
mean       7421.57
std       43079.37
min           0.00
25%           3.77
50%          74.98
75%         798.64
max      744444.41
Name: unit_water_consumption, dtype: float64

In [58]:
nic_sub_frame = asi_final.groupby(['state_code', 'nic_sub_class', 'dsl'])['input_sl_no'].count().reset_index().drop('input_sl_no', axis=1)

water_used = nic_sub_frame.merge(water_used, on='dsl', how='outer', validate='1:1')

water_used

,state_code,nic_sub_class,dsl,output_manuf_metric_converted,input_qty_consumed,unit_water_consumption
0,01,35101,100006,1091170.640,3.974902e+09,3642.786476
1,01,10304,100016,3662469.990,3.282010e+06,0.896119
2,01,11011,100078,592912.990,1.163322e+07,19.620451
3,01,11045,100085,8079828.810,6.247055e+07,7.731668
4,01,11041,100087,1765971.270,8.093000e+04,0.045827
...,...,...,...,...,...,...
465,36,10505,221805,5426.535,4.268996e+08,78668.909350
466,36,11049,222173,203059.810,8.246380e+06,40.610596
467,36,11043,222175,1864483.010,4.469379e+08,239.711420
468,36,11049,222176,122461.750,8.799700e+05,7.185672


### Step 3 - Median across NIC and State

In [59]:
water_footprint = water_used.groupby(['state_code', 'nic_sub_class'])[['unit_water_consumption', 'input_qty_consumed']].median().reset_index()

water_footprint = water_footprint.merge(test_df, on=['state_code', 'nic_sub_class'], how='left', validate="1:1")

water_footprint = water_footprint.merge(nic_df[["nic_description", "nic_sub_class"]], on='nic_sub_class', how='left', validate="m:1")

water_footprint.rename(columns={'metric':'unit_water_consumption_metric'}, inplace=True)

water_footprint

,state_code,nic_sub_class,unit_water_consumption,input_qty_consumed,unit_water_consumption_metric,nic_description
0,01,10304,0.896119,3.282010e+06,cubic meter per Litres,Manufacture of fruit or vegetable juices and t...
1,01,11011,19.620451,1.163322e+07,cubic meter per Litres,"Manufacture of distilled, potable, alcoholic b..."
2,01,11041,0.045827,8.093000e+04,cubic meter per Litres,Manufacture of aerated drinks
3,01,11043,0.237971,7.325500e+04,cubic meter per Litres,Manufacture of mineral water
4,01,11045,3.865836,3.123536e+07,cubic meter per Litres,Manufacture of soft drinks
...,...,...,...,...,...,...
232,36,11031,55.417003,4.518177e+09,cubic meter per Litres,Manufacture of beer
233,36,11043,187.150280,3.082590e+08,cubic meter per Litres,Manufacture of mineral water
234,36,11044,3709.128100,1.498150e+06,cubic meter per Tonne,Manufacture of ice
235,36,11049,7.185672,8.246380e+06,cubic meter per Litres,Manufacture of other non-alcoholic beverages n...


In [60]:
water_footprint['unit_water_consumption'].describe().round(2)

count       237.00
mean      10677.85
std       42759.39
min           0.00
25%           2.89
50%          85.49
75%        1236.28
max      372236.44
Name: unit_water_consumption, dtype: float64

Adding state names

In [61]:
state_code_dict = {
    "35": "A and N Islands",
    "28": "Andhra Pradesh",
    "12": "Arunachal Pradesh",
    "18": "Assam",
    "10": "Bihar",
   "04": "Chandigarh(U.T.)",
    "22": "Chhattisgarh",
    "26": "Dadra and Nagar Haveli",
    "25": "Daman and Diu",
   "07": "Delhi",
    "30": "Goa",
    "24": "Gujarat",
   "06": "Haryana",
   "02": "Himachal Pradesh",
   "01": "Jammu and Kashmir",
    "20": "Jharkhand",
    "29": "Karnataka",
    "32": "Kerala",
    "37": "Ladakh",
    "23": "Madhya Pradesh",
    "27": "Maharashtra",
    "14": "Manipur",
    "17": "Meghalaya",
    "13": "Nagaland",
    "15": "Mizoram",
    "21": "Odisha",
    "34": "Puducherry",
   "03": "Punjab",
   "08": "Rajasthan",
    "11": "Sikkim",
    "33": "Tamil Nadu",
    "36": "Telangana",
    "16": "Tripura",
   "09": "Uttar Pradesh",
   "05": "Uttarakhand",
    "19": "West Bengal"
}

water_footprint['state'] = water_footprint['state_code'].map(state_code_dict)

## Average Cost of Water

In [62]:
water_price = asi_final.groupby(['state_code', 'nic_sub_class'])['input_rate_per_unit'].mean().reset_index()
water_price

,state_code,nic_sub_class,input_rate_per_unit
0,01,10304,775.630
1,01,11011,29.710
2,01,11041,617.840
3,01,11043,235.840
4,01,11045,29.710
...,...,...,...
232,36,11031,29.710
233,36,11043,26.945
234,36,11044,29.710
235,36,11049,149.880


In [63]:
water_footprint = water_footprint.merge(water_price, on=['state_code', 'nic_sub_class'], how='outer', validate="1:1")
water_footprint.rename(columns={'input_rate_per_unit':'unit_water_cost_INR',
                                "input_qty_consumed":'median_actual_water_used_m3'}, inplace=True)
water_footprint

,state_code,nic_sub_class,unit_water_consumption,median_actual_water_used_m3,unit_water_consumption_metric,nic_description,state,unit_water_cost_INR
0,01,10304,0.896119,3.282010e+06,cubic meter per Litres,Manufacture of fruit or vegetable juices and t...,Jammu and Kashmir,775.630
1,01,11011,19.620451,1.163322e+07,cubic meter per Litres,"Manufacture of distilled, potable, alcoholic b...",Jammu and Kashmir,29.710
2,01,11041,0.045827,8.093000e+04,cubic meter per Litres,Manufacture of aerated drinks,Jammu and Kashmir,617.840
3,01,11043,0.237971,7.325500e+04,cubic meter per Litres,Manufacture of mineral water,Jammu and Kashmir,235.840
4,01,11045,3.865836,3.123536e+07,cubic meter per Litres,Manufacture of soft drinks,Jammu and Kashmir,29.710
...,...,...,...,...,...,...,...,...
232,36,11031,55.417003,4.518177e+09,cubic meter per Litres,Manufacture of beer,Telangana,29.710
233,36,11043,187.150280,3.082590e+08,cubic meter per Litres,Manufacture of mineral water,Telangana,26.945
234,36,11044,3709.128100,1.498150e+06,cubic meter per Tonne,Manufacture of ice,Telangana,29.710
235,36,11049,7.185672,8.246380e+06,cubic meter per Litres,Manufacture of other non-alcoholic beverages n...,Telangana,149.880


In [64]:
water_footprint['average_water_cost_industry'] = water_footprint['unit_water_consumption'] * water_footprint['unit_water_cost_INR']
water_footprint['actual_water_cost_industry'] = water_footprint['median_actual_water_used_m3'] * water_footprint['unit_water_cost_INR']
water_footprint

,state_code,nic_sub_class,unit_water_consumption,median_actual_water_used_m3,unit_water_consumption_metric,nic_description,state,unit_water_cost_INR,average_water_cost_industry,actual_water_cost_industry
0,01,10304,0.896119,3.282010e+06,cubic meter per Litres,Manufacture of fruit or vegetable juices and t...,Jammu and Kashmir,775.630,695.057003,2.545625e+09
1,01,11011,19.620451,1.163322e+07,cubic meter per Litres,"Manufacture of distilled, potable, alcoholic b...",Jammu and Kashmir,29.710,582.923586,3.456230e+08
2,01,11041,0.045827,8.093000e+04,cubic meter per Litres,Manufacture of aerated drinks,Jammu and Kashmir,617.840,28.314046,5.000179e+07
3,01,11043,0.237971,7.325500e+04,cubic meter per Litres,Manufacture of mineral water,Jammu and Kashmir,235.840,56.123162,1.727646e+07
4,01,11045,3.865836,3.123536e+07,cubic meter per Litres,Manufacture of soft drinks,Jammu and Kashmir,29.710,114.853981,9.280025e+08
...,...,...,...,...,...,...,...,...,...,...
232,36,11031,55.417003,4.518177e+09,cubic meter per Litres,Manufacture of beer,Telangana,29.710,1646.439160,1.342350e+11
233,36,11043,187.150280,3.082590e+08,cubic meter per Litres,Manufacture of mineral water,Telangana,26.945,5042.764286,8.306040e+09
234,36,11044,3709.128100,1.498150e+06,cubic meter per Tonne,Manufacture of ice,Telangana,29.710,110198.195865,4.451004e+07
235,36,11049,7.185672,8.246380e+06,cubic meter per Litres,Manufacture of other non-alcoholic beverages n...,Telangana,149.880,1076.988558,1.235967e+09


In [65]:
water_footprint.columns

Index(['state_code', 'nic_sub_class', 'unit_water_consumption',
       'median_actual_water_used_m3', 'unit_water_consumption_metric',
       'nic_description', 'state', 'unit_water_cost_INR',
       'average_water_cost_industry', 'actual_water_cost_industry'],
      dtype='object')

In [66]:
water_footprint = water_footprint[['state', 'state_code', 'nic_sub_class', 'unit_water_consumption',
        'unit_water_consumption_metric','median_actual_water_used_m3',
       'unit_water_cost_INR', 'average_water_cost_industry', 
       'actual_water_cost_industry', 'nic_description']]

water_footprint

,state,state_code,nic_sub_class,unit_water_consumption,unit_water_consumption_metric,median_actual_water_used_m3,unit_water_cost_INR,average_water_cost_industry,actual_water_cost_industry,nic_description
0,Jammu and Kashmir,01,10304,0.896119,cubic meter per Litres,3.282010e+06,775.630,695.057003,2.545625e+09,Manufacture of fruit or vegetable juices and t...
1,Jammu and Kashmir,01,11011,19.620451,cubic meter per Litres,1.163322e+07,29.710,582.923586,3.456230e+08,"Manufacture of distilled, potable, alcoholic b..."
2,Jammu and Kashmir,01,11041,0.045827,cubic meter per Litres,8.093000e+04,617.840,28.314046,5.000179e+07,Manufacture of aerated drinks
3,Jammu and Kashmir,01,11043,0.237971,cubic meter per Litres,7.325500e+04,235.840,56.123162,1.727646e+07,Manufacture of mineral water
4,Jammu and Kashmir,01,11045,3.865836,cubic meter per Litres,3.123536e+07,29.710,114.853981,9.280025e+08,Manufacture of soft drinks
...,...,...,...,...,...,...,...,...,...,...
232,Telangana,36,11031,55.417003,cubic meter per Litres,4.518177e+09,29.710,1646.439160,1.342350e+11,Manufacture of beer
233,Telangana,36,11043,187.150280,cubic meter per Litres,3.082590e+08,26.945,5042.764286,8.306040e+09,Manufacture of mineral water
234,Telangana,36,11044,3709.128100,cubic meter per Tonne,1.498150e+06,29.710,110198.195865,4.451004e+07,Manufacture of ice
235,Telangana,36,11049,7.185672,cubic meter per Litres,8.246380e+06,149.880,1076.988558,1.235967e+09,Manufacture of other non-alcoholic beverages n...


### Exporting Frames

In [67]:
asi_final_path = dir_path.joinpath("data/asi/processed/asi_final.xlsx")
water_footprint_path = dir_path.joinpath("data/asi/processed/water_footprint.xlsx")

In [68]:
# asi_final.to_excel(asi_final_path, index=False)
water_footprint.to_excel(water_footprint_path, index = False)

In [69]:
asi_final[
    (asi_final['state_code']=="36")
    &
    (asi_final['nic_sub_class']=='10505')
][['dsl','output_unit_qty_code','output_standard_metric', 'output_qty_manuf', 'metric_conversion_factor', 'output_manuf_metric_converted', 'input_qty_consumed']]

,dsl,output_unit_qty_code,output_standard_metric,output_qty_manuf,metric_conversion_factor,output_manuf_metric_converted,input_qty_consumed
628,221805,Kg,Tonne,5426535.0,0.001,5426.535,426899590.0
